# Assigment 6

This assigment will be graded if everything works well. I will run the script as once and everything should be done without errors and mistakes. I should be able to run your scripts in my computer and get all the results. **USE RELATIVE PATHS**. An error or exception or anything that breaks the code will means NO GRADE (0). Additionally, you are not able to modify any file handly. It also means NO GRADE (0). Comment everything you think will help others read your script. We expect 0 errors using GitHub. Everything will be graded!

**ASK EVERYTHING! WE ARE HERE TO HELP YOU!**

**GET YOUR GOOGLE API AND TOKEN. YOU WILL NEED THEM TO DO THIS TASK.**


1. Import Data from [this url](https://www.bbva.pe/content/dam/public-web/peru/documents/personas/canales-de-atencion/oficinas/Oficinas-BBVA-abiertas-23.10.20.pdf). This dataset is in PDF format. You have to convert to PandasDataFrame and keep only the BBVA offices in LIMA.
2. Use GoogleMaps API and geocode all the BBVA offices. For those offices that Google API gets no information, use internet and get the latitude and longitude handly and add them to dataset.
3. Use Google API to find the driving time (best guess) from all the group members' address and all the LIMA BBVA offices.
4. Finally, you have to give a report which offices are the most closest and furthest to every group member's address.

## Solution

### 7.1 Installation and import of libraries

This code read the data from de excel and import the data in a Data Frame. 

In [1]:
#Install google maps function - just at the beggining
#pip install -U googlemaps

In [2]:
import pandas as pd
import numpy as np
import os
import urllib.request, json, csv
import googlemaps
from datetime import datetime
import io
import requests
from functools import reduce
import time
from tqdm import tqdm

In [3]:
# Read data from shared excel
oficinas_bbva_lima = pd.read_excel( r"../../_data/bbva_list.xlsx" )
oficinas_bbva_lima

,Direccion,DEPARTAMENTO,PROVINCIA,DISTRITO
0,CENTRO AEREO COMERCIAL LOCALES 110 A Y 111 A,LIMA,LIMA,CALLAO
1,AV. CTRMTE. MORA S/N BASE NAVAL,LIMA,LIMA,CALLAO
2,"AV. ELMER FAUCETT Y ALEJANDRO BERTELLO, CC CAN...",LIMA,LIMA,CALLAO
3,AV. SAENZ PEN A 323,LIMA,LIMA,CALLAO
4,CALLE OMEGA 149 PARQUE INDUSTRIAL DEL CALLAO,LIMA,LIMA,CALLAO
5,AV. CONTRALMIRANTE RAYGADA N°lll,LIMA,LIMA,CALLAO
6,"AV. ELMERT FAUCETT N°2121 LOCALES N° 2-101,2-1...",LIMA,LIMA,CALLAO
7,AV. ELMER FAUCETT 6000,LIMA,LIMA,CALLAO
8,CENTRO COMERCIAL MINKA PABELL6N 2,LIMA,LIMA,CALLAO
9,"AV. OSCAR R. BENAVIDES 3866, URB. EL AGUILA, L...",LIMA,LIMA,CALLAO


### 7.2 GoogleMaps API and geocode all the BBVA offices

This code geocodes all BBVA branches with the GoogleMaps API. Also in cases without data, the code looks for latitude and longitude and adds them to the dataset.

In [4]:
# Data you want to add in the new column
oficinas = [
    "AEROPUERTO", "BASE NAVAL", "C.C. CANTA CALLAO", "CALLAO", "COLONIAL",
    "EL PUERTO", "LIMA CARGO CITY", "OVALO 200 MILLAS", "MINKA",
    "MALL AVENTURA PLAZA CALLAO", "BELLAVISTA", "GUARDIA CHALACA", "REPSOL",
    "VENTANILLA", "BACKUS", "SALAMANCA", "NICOLAS AYLLON", "PURUCHUCO",
    "C.C REAL PLAZA SANTA CLARA", "VITARTE", "HUAYCAN", "BARRANCO",
    "PLAZA BUTTERS", "VENEZUELA I", "VENEZUELA III", "BREÑA", "CARABAYLLO",
    "CEDROS DE VILLA", "CHORRILLOS", "CLUB REGATAS LIMA", "PLAZA LIMA SUR",
    "VILLA MARINA", "COMAS", "MALL PLAZA COMAS", "SAN FELIPE COMAS",
    "V. A. BELAUNDE", "EL AGUSTINO", "MEGA PLAZA", "MULTIPLAZA",
    "PLAZA LIMA NORTE", "ESSALUD", "PLAZA SAN JOSÉ", "JESUS MARÍA",
    "REAL PLAZA SALAVERRY", "SAN FELIPE", "CAMACHO", "LA PLANICIE",
    "EL REMANSO", "LA MOLINA", "LA ROTONDA", "RINCONADA", "SEPARADORA INDUSTRIAL"
]


# Construct the new list with the desired strings
oficinas = ['Oficina BBVA ' + nombre for nombre in oficinas]

# Add the new column to the DataFrame
oficinas_bbva_lima['oficinas'] = oficinas
oficinas_bbva_lima

# Rearrange columns to place 'offices' at the top
column_order = ['oficinas'] + [col for col in oficinas_bbva_lima.columns if col != 'oficinas']
oficinas_bbva_lima = oficinas_bbva_lima[column_order]

oficinas_bbva_lima

,oficinas,Direccion,DEPARTAMENTO,PROVINCIA,DISTRITO
0,Oficina BBVA AEROPUERTO,CENTRO AEREO COMERCIAL LOCALES 110 A Y 111 A,LIMA,LIMA,CALLAO
1,Oficina BBVA BASE NAVAL,AV. CTRMTE. MORA S/N BASE NAVAL,LIMA,LIMA,CALLAO
2,Oficina BBVA C.C. CANTA CALLAO,"AV. ELMER FAUCETT Y ALEJANDRO BERTELLO, CC CAN...",LIMA,LIMA,CALLAO
3,Oficina BBVA CALLAO,AV. SAENZ PEN A 323,LIMA,LIMA,CALLAO
4,Oficina BBVA COLONIAL,CALLE OMEGA 149 PARQUE INDUSTRIAL DEL CALLAO,LIMA,LIMA,CALLAO
5,Oficina BBVA EL PUERTO,AV. CONTRALMIRANTE RAYGADA N°lll,LIMA,LIMA,CALLAO
6,Oficina BBVA LIMA CARGO CITY,"AV. ELMERT FAUCETT N°2121 LOCALES N° 2-101,2-1...",LIMA,LIMA,CALLAO
7,Oficina BBVA OVALO 200 MILLAS,AV. ELMER FAUCETT 6000,LIMA,LIMA,CALLAO
8,Oficina BBVA MINKA,CENTRO COMERCIAL MINKA PABELL6N 2,LIMA,LIMA,CALLAO
9,Oficina BBVA MALL AVENTURA PLAZA CALLAO,"AV. OSCAR R. BENAVIDES 3866, URB. EL AGUILA, L...",LIMA,LIMA,CALLAO


In [5]:
import numpy as np
import googlemaps
from googlemaps.exceptions import ApiError

# Replace 'YOUR_API_KEY' with your Google Maps API Key
gmaps = googlemaps.Client(key='CLAVE API')


# Function for obtaining latitude and longitude coordinates
def get_coordenadas(row):
    direccion_detallada = f"{row['oficinas']}, {row['DEPARTAMENTO']}, {row['PROVINCIA']}, {row['DISTRITO']}, {row['Direccion']}"
    try:
        geocode_result = gmaps.geocode(direccion_detallada)
        latitud = geocode_result[0]['geometry']['location']['lat']
        longitud = geocode_result[0]['geometry']['location']['lng']
        return latitud, longitud
    except googlemaps.exceptions.ApiError as e:
        print(f"Error al obtener coordenadas: {e}")
        return np.nan, np.nan

# Create the column 'Direccion_detallada'.
oficinas_bbva_lima['Direccion_detallada'] = oficinas_bbva_lima.apply(
    lambda row: f"{row['oficinas']}, {row['DEPARTAMENTO']}, {row['PROVINCIA']}, {row['DISTRITO']}, {row['Direccion']}", axis=1)

# Get coordinates and add columns 'latitud' and 'longitud'
oficinas_bbva_lima[['latitud_oficina', 'longitud_oficina']] = oficinas_bbva_lima.apply(
    get_coordenadas, axis=1, result_type='expand')

# Show the resulting DataFrame
oficinas_bbva_lima

/var/folders/wt/06417njn63n8t03ry7nyf_wh0000gn/T/ipykernel_49294/1064612330.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oficinas_bbva_lima['Direccion_detallada'] = oficinas_bbva_lima.apply(
/var/folders/wt/06417njn63n8t03ry7nyf_wh0000gn/T/ipykernel_49294/1064612330.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oficinas_bbva_lima[['latitud_oficina', 'longitud_oficina']] = oficinas_bbva_lima.apply(
/var/folders/wt/06417njn63n8t03ry7nyf_wh0000gn/T/ipykernel_49294/1064612330.py:26: SettingWithCo

,oficinas,Direccion,DEPARTAMENTO,PROVINCIA,DISTRITO,Direccion_detallada,latitud_oficina,longitud_oficina
0,Oficina BBVA AEROPUERTO,CENTRO AEREO COMERCIAL LOCALES 110 A Y 111 A,LIMA,LIMA,CALLAO,"Oficina BBVA AEROPUERTO, LIMA, LIMA, CALLAO, C...",-12.023538,-77.104397
1,Oficina BBVA BASE NAVAL,AV. CTRMTE. MORA S/N BASE NAVAL,LIMA,LIMA,CALLAO,"Oficina BBVA BASE NAVAL, LIMA, LIMA, CALLAO, A...",-12.042604,-77.136230
2,Oficina BBVA C.C. CANTA CALLAO,"AV. ELMER FAUCETT Y ALEJANDRO BERTELLO, CC CAN...",LIMA,LIMA,CALLAO,"Oficina BBVA C.C. CANTA CALLAO, LIMA, LIMA, CA...",-11.997795,-77.112969
3,Oficina BBVA CALLAO,AV. SAENZ PEN A 323,LIMA,LIMA,CALLAO,"Oficina BBVA CALLAO, LIMA, LIMA, CALLAO, AV. S...",-12.060078,-77.137023
4,Oficina BBVA COLONIAL,CALLE OMEGA 149 PARQUE INDUSTRIAL DEL CALLAO,LIMA,LIMA,CALLAO,"Oficina BBVA COLONIAL, LIMA, LIMA, CALLAO, CAL...",-12.050680,-77.087137
5,Oficina BBVA EL PUERTO,AV. CONTRALMIRANTE RAYGADA N°lll,LIMA,LIMA,CALLAO,"Oficina BBVA EL PUERTO, LIMA, LIMA, CALLAO, AV...",-12.044327,-77.138549
6,Oficina BBVA LIMA CARGO CITY,"AV. ELMERT FAUCETT N°2121 LOCALES N° 2-101,2-1...",LIMA,LIMA,CALLAO,"Oficina BBVA LIMA CARGO CITY, LIMA, LIMA, CALL...",-12.037971,-77.098635
7,Oficina BBVA OVALO 200 MILLAS,AV. ELMER FAUCETT 6000,LIMA,LIMA,CALLAO,"Oficina BBVA OVALO 200 MILLAS, LIMA, LIMA, CAL...",-11.996881,-77.124234
8,Oficina BBVA MINKA,CENTRO COMERCIAL MINKA PABELL6N 2,LIMA,LIMA,CALLAO,"Oficina BBVA MINKA, LIMA, LIMA, CALLAO, CENTRO...",-12.048642,-77.110899
9,Oficina BBVA MALL AVENTURA PLAZA CALLAO,"AV. OSCAR R. BENAVIDES 3866, URB. EL AGUILA, L...",LIMA,LIMA,CALLAO,"Oficina BBVA MALL AVENTURA PLAZA CALLAO, LIMA,...",-12.055129,-77.102614


# Check if there are NAs

There are no NAs

In [6]:
# Check for NaN in the columns 'latitud_oficina' and 'longitud_oficina'
nan_check = oficinas_bbva_lima[['latitud_oficina', 'longitud_oficina']].isna().any()
nan_check

latitud_oficina     False
longitud_oficina    False
dtype: bool

# 7.3 Estimation of Driving Time from the Addresses of the Group Members to the Lima Offices of BBVA

This code calculates the estimated driving time and distance for each member of the group in a predefined dictionary (miembros) to the BBVA offices in the excel. Then the results are stored in a Pandas DataFrame (tiempo_conduccion_df). It uses the Google Maps API for obtaining driving directions and handles errors that may occur during the API request. The resulting DataFrame is then displayed.

In [ ]:
# Data for the members: the code defines a dictionary named "miembros" containing information about each member of the group. 
#Each member has details such as address, department, province, and district.
miembros = {
    'Claudia': {'Direccion': 'Av. Los Diamantes 734, San Martín de Porres 15109', 'DEPARTAMENTO': 'LIMA', 'PROVINCIA': 'LIMA', 'DISTRITO': 'SAN MARTIN DE PORRES'},
    'Yanell': {'Direccion': 'Jr. Mariscal Luzuriaga, 244', 'DEPARTAMENTO': 'LIMA', 'PROVINCIA': 'LIMA', 'DISTRITO': 'JESUS MARIA'},
    'Francisco': {'Direccion': 'Av. Antonio Miró Quesada 434', 'DEPARTAMENTO': 'LIMA', 'PROVINCIA': 'LIMA', 'DISTRITO': 'JESUS MARIA'},
    'Adriana': {'Direccion': 'Jirón Costa Rica 126', 'DEPARTAMENTO': 'LIMA', 'PROVINCIA': 'LIMA', 'DISTRITO': 'JESUS MARIA'},
    'Antuané': {'Direccion': 'Av. 10 de junio 1010, condominio Los Cipreses', 'DEPARTAMENTO': 'LIMA', 'PROVINCIA': 'LIMA', 'DISTRITO': 'SAN MARTIN DE PORRES'}
}

# Function to obtain the estimated driving time
def obtener_tiempo_conduccion(origen, destino):
    try:
        #Request driving directions from Google Maps API
        directions_result = gmaps.directions(origen, destino, mode="driving", departure_time=datetime.now())
        #Extract relevant information from the API response
        distancia_km = directions_result[0]['legs'][0]['distance']['text']
        distancia_metros = directions_result[0]['legs'][0]['distance']['value']
        duracion_minutos = directions_result[0]['legs'][0]['duration']['text']
        duracion_segundos = directions_result[0]['legs'][0]['duration']['value']
        duracion_trafico_minutos = directions_result[0]['legs'][0]['duration_in_traffic']['text']
        duracion_trafico_segundos = directions_result[0]['legs'][0]['duration_in_traffic']['value']
        
        # Return the obtained results
        return distancia_km, distancia_metros, duracion_minutos, duracion_segundos, duracion_trafico_minutos, duracion_trafico_segundos
    except googlemaps.exceptions.ApiError as e:
        # Handle errors from the Google Maps API
        print(f"Error al obtener la estimación del tiempo de conducción: {e}")
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    
# Create a DataFrame to store the driving time information
tiempo_conduccion_df = pd.DataFrame(columns=['Miembro', 'Oficina', 'Distancia_Km', 'Distancia_Metros', 'Duracion_Minutos', 'Duracion_Segundos', 'Duracion_Trafico_Minutos', 'Duracion_Trafico_Segundos'])

# Iterate over the members and all BBVA offices in Lima
for miembro, datos_miembro in tqdm(miembros.items(), desc='Procesando miembros'):
    # Construct the origin address using member's details
    origen = f"{datos_miembro['Direccion']}, {datos_miembro['DEPARTAMENTO']}, {datos_miembro['PROVINCIA']}, {datos_miembro['DISTRITO']}"
    
    # Iterate over all BBVA offices in Lima
    for index, datos_oficina in oficinas_bbva_lima.iterrows():
         # Extract the destination address from the BBVA offices data
        destino = datos_oficina['Direccion_detallada']
        
        # Obtain driving time information for the current combination of member and office
        distancia_km, distancia_metros, duracion_minutos, duracion_segundos, duracion_trafico_minutos, duracion_trafico_segundos = obtener_tiempo_conduccion(origen, destino)
        
         # Create a temporary DataFrame with the current information
        temp_df = pd.DataFrame({
            'Miembro': [miembro],
            'Oficina': [datos_oficina['oficinas']],
            'Distancia_Km': [distancia_km],
            'Distancia_Metros': [distancia_metros],
            'Duracion_Minutos': [duracion_minutos],
            'Duracion_Segundos': [duracion_segundos],
            'Duracion_Trafico_Minutos': [duracion_trafico_minutos],
            'Duracion_Trafico_Segundos': [duracion_trafico_segundos]
        })

        # Concatenate the temporary DataFrame to the main DataFrame
        tiempo_conduccion_df = pd.concat([tiempo_conduccion_df, temp_df], ignore_index=True)

# Display the DataFrame with the driving time information
tiempo_conduccion_df

In [ ]:
# Then we proceed to evaluate the distances of the offices for each member of the group by accessing the DataFrame of each one

### Miembro 1: Claudia

In [ ]:
# Access the DataFrame for the first member
# Extract results for a specific member ('Claudia') from the driving time DataFrame
df_miembro_1_resultados = tiempo_conduccion_df[tiempo_conduccion_df['Miembro'] == 'Claudia']
# Display the DataFrame with driving time results for the specific member
df_miembro_1_resultados

### Miembro 2: Yanell

In [ ]:
# Access the DataFrame for the second member
# Extract results for a specific member ('Yanell') from the driving time DataFrame
df_miembro_2_resultados = tiempo_conduccion_df[tiempo_conduccion_df['Miembro'] == 'Yanell']
# Display the DataFrame with driving time results for the specific member
df_miembro_2_resultados

### Miembro 3: Francisco

In [ ]:
# Access the DataFrame for the third member
# Extract results for a specific member ('Francisco') from the driving time DataFrame
df_miembro_3_resultados = tiempo_conduccion_df[tiempo_conduccion_df['Miembro'] == 'Francisco']
# Display the DataFrame with driving time results for the specific member
df_miembro_3_resultados

### Miembro 4: Adriana

In [ ]:
#Access the DataFrame for the fourth member
# Extract results for a specific member ('Adriana') from the driving time DataFrame
df_miembro_4_resultados = tiempo_conduccion_df[tiempo_conduccion_df['Miembro'] == 'Adriana']
# Display the DataFrame with driving time results for the specific member
df_miembro_4_resultados

### Miembro 5: Antuané

In [ ]:
#Access the DataFrame for the fifth member
# Extract results for a specific member ('Antuané') from the driving time DataFrame
df_miembro_5_resultados = tiempo_conduccion_df[tiempo_conduccion_df['Miembro'] == 'Antuané']
# Display the DataFrame with driving time results for the specific member
df_miembro_5_resultados